In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install library yang diperlukan
!pip install fastai --upgrade timm --upgrade

In [ ]:
# Import libraries
from fastai.vision.all import *
import pandas as pd
import numpy as np
import random
from timm import create_model
from torchvision.transforms import ColorJitter, RandomErasing
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Set random seed untuk konsistensi
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)

In [ ]:
# Load train.csv
train_df = pd.read_csv('/content/drive/MyDrive/HOLOGY/training.csv')

In [ ]:
# Mendefinisikan path ke gambar
path = Path('/content/drive/MyDrive/HOLOGY/train')

In [ ]:
# Fungsi untuk mencari ekstensi file gambar
def get_image_filename(image_id, path):
    for ext in ['.jpg', '.png']:
        file = path / f'{image_id}{ext}'
        if file.exists():
            return file.name
    raise FileNotFoundError(f"Image {image_id} not found.")

In [ ]:
# Update kolom 'id' dengan nama file yang benar
train_df['id'] = train_df['id'].apply(lambda x: get_image_filename(x, path))

In [ ]:
# Pastikan tidak ada nilai kosong
train_df = train_df.dropna(subset=['jenis', 'warna'])

In [ ]:
# Ubah pipeline data untuk menggunakan aug_transforms dari FastAI
item_tfms = [Resize(224)]  # Resize dilakukan di awal

# Gunakan batch_tfms untuk augmentasi yang lebih kompleks dan pastikan format data sesuai
batch_tfms = aug_transforms(
    do_flip=True, flip_vert=True, max_zoom=1.1,
    max_rotate=20, max_lighting=0.4, max_warp=0.2,
    p_affine=0.8, p_lighting=0.8
) + [Normalize.from_stats(*imagenet_stats)]

In [ ]:
# DataBlock untuk label 'jenis'
data_block_jenis = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_x=ColReader('id', pref=str(path) + "/"),
    get_y=ColReader('jenis'),
    item_tfms=item_tfms,
    batch_tfms=batch_tfms
)

In [ ]:
# DataLoader untuk 'jenis' dengan batch size 32
dls_jenis = data_block_jenis.dataloaders(train_df, bs=32)

In [ ]:
# Model for 'jenis'
model_jenis = create_model('resnet152', pretrained=True, num_classes=len(dls_jenis.vocab))

# Learner for 'jenis'
learn_jenis = Learner(
    dls_jenis, model_jenis, metrics=accuracy,
    cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=3)],
    wd=1e-2
)

In [ ]:
# Fine-tuning 'jenis' dengan lebih dalam
learn_jenis.freeze()
learn_jenis.fit_one_cycle(5, 1e-3)
learn_jenis.unfreeze()
learn_jenis.fit_one_cycle(10, lr_max=slice(1e-5, 1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.855931,0.239921,0.909677,00:15
1,0.490352,0.141889,0.948387,00:14
2,0.326433,0.115519,0.961290,00:11
3,0.230707,0.117582,0.954839,00:11
4,0.177360,0.130859,0.941935,00:11


epoch,train_loss,valid_loss,accuracy,time
0,0.038252,0.123902,0.954839,00:12
1,0.076250,0.143313,0.948387,00:11
2,0.085189,0.090832,0.967742,00:11
3,0.092623,0.147955,0.941935,00:12
4,0.099489,0.119653,0.954839,00:12
5,0.079809,0.140043,0.954839,00:11


No improvement since epoch 2: early stopping


In [ ]:
# Stochastic Weight Averaging (SWA) untuk model jenis
class SWACallback(Callback):
    def __init__(self):
        self.n_models = 0
        self.swa_weights = None

    def after_epoch(self):
        if self.n_models == 0:
            self.swa_weights = {k: v.detach().clone() for k, v in self.learn.model.state_dict().items()}
        else:
            for key in self.learn.model.state_dict().keys():
                self.swa_weights[key] += self.learn.model.state_dict()[key].detach().clone()
        self.n_models += 1

    def after_fit(self):
        for key in self.learn.model.state_dict().keys():
            # Pastikan semua tensor dalam swa_weights adalah float
            self.swa_weights[key] = self.swa_weights[key].float() / self.n_models
        self.learn.model.load_state_dict(self.swa_weights)

In [ ]:
# Training dengan SWA
learn_jenis.fit_one_cycle(5, lr_max=slice(1e-6, 1e-4), cbs=[SWACallback()])

epoch,train_loss,valid_loss,accuracy,time
0,0.039501,0.127548,0.954839,00:14
1,0.033736,0.121277,0.961290,00:16
2,0.035951,0.123031,0.954839,00:13
3,0.026949,0.121641,0.954839,00:13
4,0.028984,0.122266,0.954839,00:13


No improvement since epoch 1: early stopping


In [ ]:
# DataBlock untuk label 'warna'
data_block_warna = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_x=ColReader('id', pref=str(path) + "/"),
    get_y=ColReader('warna'),
    item_tfms=item_tfms,
    batch_tfms=batch_tfms
)

In [ ]:
# DataLoader untuk 'warna' dengan batch size 32
dls_warna = data_block_warna.dataloaders(train_df, bs=32)

In [ ]:
# Model for 'warna'
model_warna = create_model('densenet121', pretrained=True, num_classes=len(dls_warna.vocab))

# Learner for 'warna'
learn_warna = Learner(
    dls_warna, model_warna, metrics=accuracy,
    cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=5)],
    wd=1e-2
)

In [ ]:
# Fine-tuning 'warna' dengan lebih dalam
learn_warna.freeze()
learn_warna.fit_one_cycle(5, 1e-3)
learn_warna.unfreeze()
learn_warna.fit_one_cycle(10, lr_max=slice(1e-5, 1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.583305,0.757434,0.741935,00:11
1,0.814196,0.144809,0.961290,00:11
2,0.493906,0.145668,0.967742,00:11
3,0.313156,0.127034,0.961290,00:12
4,0.210752,0.119939,0.974194,00:11


epoch,train_loss,valid_loss,accuracy,time
0,0.041730,0.103109,0.980645,00:11
1,0.056693,0.147448,0.961290,00:11
2,0.087832,0.119276,0.974194,00:11
3,0.067993,0.116463,0.987097,00:11
4,0.061708,0.155222,0.967742,00:11
5,0.058772,0.174777,0.967742,00:11


No improvement since epoch 0: early stopping


In [ ]:
# Stochastic Weight Averaging (SWA) untuk model warna
learn_warna.fit_one_cycle(5, lr_max=slice(1e-6, 1e-4), cbs=[SWACallback()])

epoch,train_loss,valid_loss,accuracy,time
0,0.016763,0.141701,0.967742,00:11
1,0.039185,0.128558,0.974194,00:13
2,0.030387,0.124978,0.974194,00:13
3,0.024466,0.124197,0.974194,00:15
4,0.024509,0.127756,0.980645,00:13


In [ ]:
# Mempersiapkan data test untuk prediksi
test_path = Path('/content/drive/MyDrive/HOLOGY/test')
test_image = get_image_files(test_path)

In [ ]:
# Tambahkan data test ke DataLoader untuk jenis
test_dl_jenis = learn_jenis.dls.test_dl(test_image)

In [ ]:
# Lakukan TTA pada DataLoader pengujian untuk jenis
tta_preds_jenis, _ = learn_jenis.tta(dl=test_dl_jenis, n=5)
final_classes_jenis = [dls_jenis.vocab[i] for i in torch.argmax(tta_preds_jenis, dim=1)]

In [ ]:
# Tambahkan data test ke DataLoader untuk warna
test_dl_warna = learn_warna.dls.test_dl(test_image)

In [ ]:
# Lakukan TTA pada DataLoader pengujian untuk warna
tta_preds_warna, _ = learn_warna.tta(dl=test_dl_warna, n=5)
final_classes_warna = [dls_warna.vocab[i] for i in torch.argmax(tta_preds_warna, dim=1)]

In [ ]:
# Buat DataFrame untuk hasil prediksi
submission_df = pd.DataFrame({
    'id': [int(f.stem) for f in test_image],
    'jenis': [jenis for jenis in final_classes_jenis],
    'warna': [warna for warna in final_classes_warna]
})

In [ ]:
# Urutkan berdasarkan 'id' dan simpan ke CSV
submission_df = submission_df.sort_values(by='id')
submission_df.to_csv('submission_resnetdensenet.csv', index=False)

In [ ]:
# Download hasil submission
from google.colab import files
files.download('submission_resnetdensenet.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>